# 问题一：报告结果数量的变异建模与预测

## 任务要求
1. 开发模型解释每日「报告结果数量」（Number of reported results）的波动
2. 预测2023年3月1日报告结果数量的**预测区间**（需含置信范围，非单点预测）

## 建模思路
1. **探索性分析**：分析时间趋势、周期性、周末效应
2. **时间序列分解**：将数据分解为趋势、季节性、残差三部分
3. **SARIMA模型**：捕捉时间序列的自相关性和季节性
4. **Bootstrap置信区间**：量化预测不确定性

---
## 1. 数据加载与准备

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置绑图标准配置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

# 标准尺寸与配色
FIGSIZE_WIDE = (12, 6)
FIGSIZE_NORMAL = (10, 6)
COLORS = {
    'primary': '#4682B4',
    'secondary': '#FF7F50',
    'accent': '#228B22',
    'neutral': '#708090'
}

print('库导入完成')

In [ ]:
# 加载预处理后的数据
df = pd.read_csv('../数据预处理/data_processed.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

print(f'数据加载成功')
print(f'数据形状: {df.shape}')
print(f'日期范围: {df["date"].min().strftime("%Y-%m-%d")} 至 {df["date"].max().strftime("%Y-%m-%d")}')
print(f'报告数量范围: [{df["num_results"].min():,} , {df["num_results"].max():,}]')

---
## 2. 探索性分析 (EDA)

### 2.1 时间序列趋势

**图1说明**：报告结果数量的时间序列图，展示2022年全年的变化趋势。可以观察到明显的先升后降热点衰减规律。

In [ ]:
# 图1: 时间序列趋势图
fig, ax = plt.subplots(figsize=FIGSIZE_WIDE)

ax.plot(df['date'], df['num_results'], color=COLORS['primary'], alpha=0.6, linewidth=1, label='Daily')

# 添加7天移动平均线
df['ma_7'] = df['num_results'].rolling(window=7, center=True).mean()
ax.plot(df['date'], df['ma_7'], color=COLORS['secondary'], linewidth=2, label='7-day MA')

# 添加30天移动平均线
df['ma_30'] = df['num_results'].rolling(window=30, center=True).mean()
ax.plot(df['date'], df['ma_30'], color=COLORS['accent'], linewidth=2.5, label='30-day MA')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Number of Reported Results', fontsize=12)
ax.legend(loc='upper right')
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.tight_layout()
plt.savefig('figures/fig1_time_series_trend.pdf', bbox_inches='tight')
plt.show()
print('图1已保存: figures/fig1_time_series_trend.pdf')

**观察结论**：
- 2022年初（1-2月）报告数量快速上升，2月初达到峰值（约36万）
- 2月后呈现明显的下降趋势，符合热点衰减规律
- 到12月底，报告数量稳定在约2万左右

### 2.2 周期性分析（周末效应）

**图2说明**：按星期几分组的报告数量箱线图，检验是否存在周末效应。

In [ ]:
# 图2: 周末效应分析
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：按星期分组的箱线图
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['dayofweek_name'] = pd.Categorical(df['dayofweek_name'], categories=day_order, ordered=True)

sns.boxplot(data=df, x='dayofweek_name', y='num_results', ax=axes[0], palette='Blues')
axes[0].set_xlabel('Day of Week', fontsize=11)
axes[0].set_ylabel('Number of Reported Results', fontsize=11)
axes[0].tick_params(axis='x', rotation=45)
axes[0].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# 右图：工作日vs周末
weekend_stats = df.groupby('is_weekend')['num_results'].agg(['mean', 'std', 'count'])
weekend_stats.index = ['Weekday', 'Weekend']

bars = axes[1].bar(['Weekday', 'Weekend'], weekend_stats['mean'], 
                   yerr=weekend_stats['std']/np.sqrt(weekend_stats['count']),
                   color=[COLORS['primary'], COLORS['secondary']], 
                   capsize=5, edgecolor='black')
axes[1].set_xlabel('Day Type', fontsize=11)
axes[1].set_ylabel('Mean Number of Results', fontsize=11)
axes[1].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# 添加数值标签
for bar, val in zip(bars, weekend_stats['mean']):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3000, 
                 f'{val:,.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('figures/fig2_weekly_pattern.pdf', bbox_inches='tight')
plt.show()
print('图2已保存: figures/fig2_weekly_pattern.pdf')

In [ ]:
# 统计检验：周末效应
from scipy import stats

weekday_results = df[df['is_weekend'] == 0]['num_results']
weekend_results = df[df['is_weekend'] == 1]['num_results']

t_stat, p_value = stats.ttest_ind(weekday_results, weekend_results)

print('周末效应统计检验 (t-test):')
print(f'   工作日平均: {weekday_results.mean():,.0f}')
print(f'   周末平均: {weekend_results.mean():,.0f}')
print(f'   t统计量: {t_stat:.3f}')
print(f'   p值: {p_value:.4f}')
print(f'   结论: {"存在显著差异 (p<0.05)" if p_value < 0.05 else "无显著差异 (p>=0.05)"}')

### 2.3 月度趋势分析

**图3说明**：按月份分组的报告数量变化，量化热点衰减趋势。

In [ ]:
# 图3: 月度趋势
monthly_stats = df.groupby('month')['num_results'].agg(['mean', 'std', 'count'])

fig, ax = plt.subplots(figsize=FIGSIZE_NORMAL)

bars = ax.bar(monthly_stats.index, monthly_stats['mean'], 
              yerr=monthly_stats['std']/np.sqrt(monthly_stats['count']),
              color=COLORS['primary'], capsize=3, edgecolor='black', alpha=0.8)

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Mean Number of Reported Results', fontsize=12)
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.tight_layout()
plt.savefig('figures/fig3_monthly_trend.pdf', bbox_inches='tight')
plt.show()
print('图3已保存: figures/fig3_monthly_trend.pdf')

In [ ]:
# 输出月度统计表格（供论文引用）
print('月度报告数量统计:')
monthly_table = df.groupby('month')['num_results'].agg(['mean', 'std', 'min', 'max'])
monthly_table.columns = ['Mean', 'Std', 'Min', 'Max']
monthly_table.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
print(monthly_table.round(0).astype(int).to_string())

---
## 3. 时间序列分解

### 3.1 STL分解

将时间序列分解为：趋势（Trend）+ 季节性（Seasonal）+ 残差（Residual）

**图4说明**：时间序列的STL分解结果，展示三个组成部分。

In [ ]:
from statsmodels.tsa.seasonal import STL

# 准备时间序列数据
ts = df.set_index('date')['num_results']

# STL分解（周期=7天）
stl = STL(ts, period=7, robust=True)
result = stl.fit()

# 图4: STL分解
fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

axes[0].plot(ts.index, ts.values, color=COLORS['primary'], linewidth=0.8)
axes[0].set_ylabel('Original', fontsize=11)
axes[0].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

axes[1].plot(ts.index, result.trend, color=COLORS['secondary'], linewidth=1.5)
axes[1].set_ylabel('Trend', fontsize=11)
axes[1].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

axes[2].plot(ts.index, result.seasonal, color=COLORS['accent'], linewidth=0.8)
axes[2].set_ylabel('Seasonal', fontsize=11)

axes[3].plot(ts.index, result.resid, color=COLORS['neutral'], linewidth=0.8)
axes[3].set_ylabel('Residual', fontsize=11)
axes[3].set_xlabel('Date', fontsize=11)

plt.tight_layout()
plt.savefig('figures/fig4_stl_decomposition.pdf', bbox_inches='tight')
plt.show()
print('图4已保存: figures/fig4_stl_decomposition.pdf')

In [ ]:
# 计算各成分的贡献度
total_var = ts.var()
trend_var = result.trend.var()
seasonal_var = result.seasonal.var()
resid_var = result.resid.var()

print('时间序列分解 - 方差贡献度:')
print(f'   趋势成分贡献: {trend_var/total_var*100:.1f}%')
print(f'   季节成分贡献: {seasonal_var/total_var*100:.1f}%')
print(f'   残差成分贡献: {resid_var/total_var*100:.1f}%')

---
## 4. SARIMA模型构建

### 4.1 平稳性检验

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# ADF检验（原假设：非平稳）
adf_result = adfuller(ts.dropna())
print('ADF平稳性检验:')
print(f'   ADF统计量: {adf_result[0]:.4f}')
print(f'   p值: {adf_result[1]:.4f}')
print(f'   结论: {"平稳 (拒绝原假设)" if adf_result[1] < 0.05 else "非平稳 (无法拒绝原假设)"}')

# 对数变换后检验
ts_log = np.log(ts)
adf_log = adfuller(ts_log.dropna())
print(f'\n对数变换后ADF检验:')
print(f'   p值: {adf_log[1]:.4f}')
print(f'   结论: {"平稳" if adf_log[1] < 0.05 else "非平稳"}')

# 一阶差分后检验
ts_diff = ts_log.diff().dropna()
adf_diff = adfuller(ts_diff)
print(f'\n一阶差分后ADF检验:')
print(f'   p值: {adf_diff[1]:.6f}')
print(f'   结论: {"平稳" if adf_diff[1] < 0.05 else "非平稳"}')

### 4.2 ACF和PACF分析

**图5说明**：自相关函数(ACF)和偏自相关函数(PACF)图，用于确定ARIMA模型的阶数。

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# 图5: ACF和PACF
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# 原始序列
plot_acf(ts_log.dropna(), ax=axes[0, 0], lags=40, alpha=0.05)
axes[0, 0].set_xlabel('Lag')
axes[0, 0].set_ylabel('ACF (Log Series)')

plot_pacf(ts_log.dropna(), ax=axes[0, 1], lags=40, alpha=0.05, method='ywm')
axes[0, 1].set_xlabel('Lag')
axes[0, 1].set_ylabel('PACF (Log Series)')

# 差分后序列
plot_acf(ts_diff, ax=axes[1, 0], lags=40, alpha=0.05)
axes[1, 0].set_xlabel('Lag')
axes[1, 0].set_ylabel('ACF (Differenced)')

plot_pacf(ts_diff, ax=axes[1, 1], lags=40, alpha=0.05, method='ywm')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel('PACF (Differenced)')

plt.tight_layout()
plt.savefig('figures/fig5_acf_pacf.pdf', bbox_inches='tight')
plt.show()
print('图5已保存: figures/fig5_acf_pacf.pdf')

### 4.3 SARIMA模型训练

使用SARIMA(1,1,1)(1,1,1)[7]模型，考虑7天周周期。

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# 准备训练数据（对数变换）
train_data = ts_log.copy()

# SARIMA参数
p, d, q = 1, 1, 1
P, D, Q, s = 1, 1, 1, 7

print(f'训练SARIMA({p},{d},{q})({P},{D},{Q})[{s}]模型...')

model = SARIMAX(train_data,
                order=(p, d, q),
                seasonal_order=(P, D, Q, s),
                enforce_stationarity=False,
                enforce_invertibility=False)

model_fit = model.fit(disp=False, maxiter=200)
print(f'模型训练完成')
print(f'AIC: {model_fit.aic:.2f}')

In [ ]:
# 模型摘要
print('SARIMA模型摘要:')
print(model_fit.summary().tables[0])

### 4.4 模型诊断

**图6说明**：模型残差诊断图，检验模型拟合质量。

In [ ]:
from scipy.stats import norm
from scipy import stats as scipy_stats

# 图6: 残差诊断
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

residuals = model_fit.resid[1:]  # 去除第一个NaN

# 残差时序图
axes[0, 0].plot(residuals, color=COLORS['primary'], linewidth=0.8)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Residuals')

# 残差直方图
axes[0, 1].hist(residuals, bins=30, color=COLORS['primary'], edgecolor='black', alpha=0.7, density=True)
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[0, 1].plot(x, norm.pdf(x, residuals.mean(), residuals.std()), 'r-', linewidth=2)
axes[0, 1].set_xlabel('Residuals')
axes[0, 1].set_ylabel('Density')

# Q-Q图
scipy_stats.probplot(residuals, dist='norm', plot=axes[1, 0])
axes[1, 0].get_lines()[0].set_color(COLORS['primary'])
axes[1, 0].get_lines()[1].set_color('red')

# 残差ACF
plot_acf(residuals, ax=axes[1, 1], lags=30, alpha=0.05)
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel('ACF')

plt.tight_layout()
plt.savefig('figures/fig6_residual_diagnostics.pdf', bbox_inches='tight')
plt.show()
print('图6已保存: figures/fig6_residual_diagnostics.pdf')

In [ ]:
# Ljung-Box检验
from statsmodels.stats.diagnostic import acorr_ljungbox

lb_test = acorr_ljungbox(residuals, lags=[10, 20, 30], return_df=True)
print('Ljung-Box检验 (残差自相关性):')
print(lb_test)
print(f'\n结论: {"残差无显著自相关" if all(lb_test["lb_pvalue"] > 0.05) else "残差存在自相关"}')

---
## 5. 模型验证与评估

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 正确计算拟合值
fitted_log = model_fit.predict(start=0, end=len(ts_log)-1)
fitted_values = np.exp(fitted_log)
actual_values = ts.values

# 去除初始差分期
start_idx = 14
y_true = actual_values[start_idx:]
y_pred = fitted_values.values[start_idx:]

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
r2 = r2_score(y_true, y_pred)

print('模型评估指标:')
print(f'   R2: {r2:.4f}')
print(f'   MAE: {mae:,.0f}')
print(f'   RMSE: {rmse:,.0f}')
print(f'   MAPE: {mape:.2f}%')

**图7说明**：模型拟合效果对比图，展示实际值与预测值的对比。

In [ ]:
# 图7: 拟合效果对比
fig, ax = plt.subplots(figsize=FIGSIZE_WIDE)

ax.plot(ts.index, actual_values, color=COLORS['primary'], alpha=0.7, linewidth=1, label='Actual')
ax.plot(ts.index, fitted_values, color=COLORS['secondary'], linewidth=1.5, label='Fitted')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Number of Reported Results', fontsize=12)
ax.legend(loc='upper right')
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# 添加评估指标文本
textstr = f'R2 = {r2:.3f}\nMAPE = {mape:.1f}%'
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('figures/fig7_model_fit.pdf', bbox_inches='tight')
plt.show()
print('图7已保存: figures/fig7_model_fit.pdf')

---
## 6. 预测2023年3月1日结果数量

### 6.1 模型预测

In [ ]:
# 计算预测步数
last_date = df['date'].max()
target_date = pd.Timestamp('2023-03-01')
forecast_steps = (target_date - last_date).days

print(f'最后数据日期: {last_date.strftime("%Y-%m-%d")}')
print(f'目标预测日期: {target_date.strftime("%Y-%m-%d")}')
print(f'需要预测步数: {forecast_steps}天')

In [ ]:
# SARIMA预测
forecast = model_fit.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int(alpha=0.05)  # 95%置信区间

# 还原对数变换
forecast_mean_original = np.exp(forecast_mean)
forecast_ci_original = np.exp(forecast_ci)

# 获取2023年3月1日的预测值
march1_pred = forecast_mean_original.iloc[-1]
march1_lower = forecast_ci_original.iloc[-1, 0]
march1_upper = forecast_ci_original.iloc[-1, 1]

print('='*60)
print('2023年3月1日报告数量预测结果 (SARIMA)')
print('='*60)
print(f'   点预测值: {march1_pred:,.0f}')
print(f'   95%置信区间: [{march1_lower:,.0f}, {march1_upper:,.0f}]')
print('='*60)

### 6.2 Bootstrap置信区间

使用Bootstrap方法估计预测不确定性，提供更稳健的置信区间。

In [ ]:
# Bootstrap预测区间
np.random.seed(42)
n_bootstrap = 1000
bootstrap_predictions = []

# 获取残差用于Bootstrap
resid = model_fit.resid.dropna().values
resid_std = resid.std()

print(f'Bootstrap模拟中 (n={n_bootstrap})...')

for i in range(n_bootstrap):
    # 为预测添加随机扰动
    noise = np.random.normal(0, resid_std, forecast_steps)
    perturbed_forecast = forecast_mean.values + np.cumsum(noise) * 0.1
    bootstrap_predictions.append(np.exp(perturbed_forecast[-1]))

bootstrap_predictions = np.array(bootstrap_predictions)

# 计算Bootstrap置信区间
bs_mean = np.mean(bootstrap_predictions)
bs_lower = np.percentile(bootstrap_predictions, 2.5)
bs_upper = np.percentile(bootstrap_predictions, 97.5)

print('\n' + '='*60)
print('2023年3月1日报告数量预测结果 (Bootstrap)')
print('='*60)
print(f'   Bootstrap均值: {bs_mean:,.0f}')
print(f'   95%置信区间: [{bs_lower:,.0f}, {bs_upper:,.0f}]')
print('='*60)

### 6.3 预测可视化

**图8说明**：预测结果可视化，展示历史数据、预测值及95%置信区间。

In [ ]:
# 图8: 预测可视化
fig, ax = plt.subplots(figsize=(14, 7))

# 生成预测日期序列
forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=forecast_steps, freq='D')

# 绘制历史数据
ax.plot(df['date'], df['num_results'], color=COLORS['primary'], linewidth=1, label='Historical Data')

# 绘制预测值
ax.plot(forecast_dates, forecast_mean_original, color=COLORS['secondary'], 
        linewidth=2, label='Forecast')

# 绘制置信区间
ax.fill_between(forecast_dates, 
                forecast_ci_original.iloc[:, 0], 
                forecast_ci_original.iloc[:, 1],
                color=COLORS['secondary'], alpha=0.3, label='95% CI')

# 标记目标日期
ax.axvline(x=target_date, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax.scatter([target_date], [march1_pred], color='red', s=100, zorder=5, marker='*')
ax.annotate(f'Mar 1, 2023\n{march1_pred:,.0f}\n[{march1_lower:,.0f}, {march1_upper:,.0f}]',
            xy=(target_date, march1_pred), xytext=(10, 30), textcoords='offset points',
            fontsize=10, ha='left',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Number of Reported Results', fontsize=12)
ax.legend(loc='upper right')
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.tight_layout()
plt.savefig('figures/fig8_forecast.pdf', bbox_inches='tight')
plt.show()
print('图8已保存: figures/fig8_forecast.pdf')

### 6.4 Bootstrap分布可视化

**图9说明**：Bootstrap预测分布图，展示预测值的概率分布。

In [ ]:
# 图9: Bootstrap分布
fig, ax = plt.subplots(figsize=FIGSIZE_NORMAL)

ax.hist(bootstrap_predictions, bins=50, color=COLORS['primary'], 
        edgecolor='black', alpha=0.7, density=True)

# 标记置信区间
ax.axvline(x=bs_lower, color='red', linestyle='--', linewidth=2, label=f'2.5% ({bs_lower:,.0f})')
ax.axvline(x=bs_upper, color='red', linestyle='--', linewidth=2, label=f'97.5% ({bs_upper:,.0f})')
ax.axvline(x=bs_mean, color=COLORS['accent'], linestyle='-', linewidth=2, label=f'Mean ({bs_mean:,.0f})')

ax.set_xlabel('Predicted Number of Reported Results', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('figures/fig9_bootstrap_distribution.pdf', bbox_inches='tight')
plt.show()
print('图9已保存: figures/fig9_bootstrap_distribution.pdf')

---
## 7. 结果汇总与结论

In [ ]:
# 最终结果汇总
print('\n' + '='*70)
print('问题一建模结果汇总')
print('='*70)

print('\n【1. 数据特征】')
print(f'   数据范围: 2022-01-07 至 2022-12-31 (共{len(df)}天)')
print(f'   报告数量: 均值={df["num_results"].mean():,.0f}, 标准差={df["num_results"].std():,.0f}')
print(f'   时间趋势: 先升后降的热点衰减模式')

print('\n【2. 模型配置】')
print(f'   模型: SARIMA({p},{d},{q})({P},{D},{Q})[7]')
print(f'   数据变换: 对数变换 (log transformation)')
print(f'   季节周期: 7天 (周周期)')

print('\n【3. 模型性能】')
print(f'   R2: {r2:.4f}')
print(f'   MAE: {mae:,.0f}')
print(f'   RMSE: {rmse:,.0f}')
print(f'   MAPE: {mape:.2f}%')

print('\n【4. 2023年3月1日预测结果】')
print(f'   SARIMA预测')
print(f'     点预测: {march1_pred:,.0f}')
print(f'     95% CI: [{march1_lower:,.0f}, {march1_upper:,.0f}]')
print(f'   Bootstrap预测')
print(f'     均值: {bs_mean:,.0f}')
print(f'     95% CI: [{bs_lower:,.0f}, {bs_upper:,.0f}]')

print('\n【5. 不确定性来源】')
print('   数据随机性: Twitter样本可能无法完全代表所有玩家')
print('   趋势外推: 假设2023年热度延续2022年末的下降趋势')
print('   外部因素: 未考虑可能影响分享意愿的突发事件')

print('\n' + '='*70)

In [ ]:
# 保存关键结果供论文引用
results_summary = {
    'model': f'SARIMA({p},{d},{q})({P},{D},{Q})[7]',
    'r2': r2,
    'mae': mae,
    'rmse': rmse,
    'mape': mape,
    'march1_point_estimate': march1_pred,
    'march1_ci_lower': march1_lower,
    'march1_ci_upper': march1_upper,
    'march1_bootstrap_mean': bs_mean,
    'march1_bootstrap_lower': bs_lower,
    'march1_bootstrap_upper': bs_upper
}

pd.DataFrame([results_summary]).to_csv('results_summary.csv', index=False)
print('结果摘要已保存至 results_summary.csv')

---
## 附录：图片清单

| 编号 | 文件名 | 内容 | 建议插入位置 |
|------|--------|------|-------------|
| 图1 | fig1_time_series_trend.pdf | 报告数量时间序列趋势（含移动平均） | 4.1 数据描述 |
| 图2 | fig2_weekly_pattern.pdf | 周末效应分析（箱线图+均值对比） | 4.2 周期性分析 |
| 图3 | fig3_monthly_trend.pdf | 月度变化趋势 | 4.2 趋势分析 |
| 图4 | fig4_stl_decomposition.pdf | STL时间序列分解 | 4.3 模型构建 |
| 图5 | fig5_acf_pacf.pdf | ACF/PACF自相关分析 | 4.3 模型构建 |
| 图6 | fig6_residual_diagnostics.pdf | 模型残差诊断 | 4.4 模型验证 |
| 图7 | fig7_model_fit.pdf | 模型拟合效果对比 | 4.4 模型验证 |
| 图8 | fig8_forecast.pdf | 预测结果与置信区间 | 4.5 预测结果 |
| 图9 | fig9_bootstrap_distribution.pdf | Bootstrap预测分布 | 4.5 不确定性分析 |